In [1]:
#importing libraries
import pandas as pd
import nltk
import gensim
import time
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import re
import string
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to
[nltk_data]     /users/grad/umukherjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /users/grad/umukherjee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [39]:
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
from gensim import corpora,models

In [46]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus

In [36]:
from scipy.spatial import distance
import pandas as pd
import nltk
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
from gensim import corpora,models
import time
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from gensim.models import Word2Vec,FastText
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /users/grad/umukherjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /users/grad/umukherjee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#reading the pre-downloaded csv file from Bugzilla
Eclipse_data=pd.read_csv('eclipse_total_data_preprocessed.csv')
Eclipse_data.tail()

,Unnamed: 0,Bug_ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
46259,46259,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,it should be possible to update a text file us...,NaN
46260,46260,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,in the case of switchlabeledrule compiler a...,NaN
46261,46261,565646.0,Community,CI-Jenkins,CLOSED,DUPLICATE,help needed in ensuring that https://ci.eclips...,2020-08-14 09:01:13,hi are there any memory cpu usage reports fo...,565863.0
46262,46262,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,created attachment downloads for atom vscode...,NaN
46263,46263,572739.0,JDT,UI,CLOSED,WONTFIX,[quick fix] to fix reliance on default encoding,2021-10-26 18:08:29,jdt ui alone has uses of code that makes use...,NaN


In [3]:
Eclipse_data = Eclipse_data.drop(columns = ["Unnamed: 0"])
Eclipse_data.tail()

,Bug_ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
46259,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,it should be possible to update a text file us...,NaN
46260,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,in the case of switchlabeledrule compiler a...,NaN
46261,565646.0,Community,CI-Jenkins,CLOSED,DUPLICATE,help needed in ensuring that https://ci.eclips...,2020-08-14 09:01:13,hi are there any memory cpu usage reports fo...,565863.0
46262,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,created attachment downloads for atom vscode...,NaN
46263,572739.0,JDT,UI,CLOSED,WONTFIX,[quick fix] to fix reliance on default encoding,2021-10-26 18:08:29,jdt ui alone has uses of code that makes use...,NaN


In [4]:
Eclipse_data.reset_index(drop=True)

,Bug_ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
0,518088.0,Web Tools,Web Standard Tools,RESOLVED,INVALID,search widget is not working,2017-06-10 14:42:10,search widget is not working,NaN
1,546444.0,z_Archived,PDT,CLOSED,NOT_ECLIPSE,Bug,2020-05-14 11:25:40,created attachment eclipse hello world,NaN
2,533893.0,Platform,Website,RESOLVED,INVALID,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,2018-04-22 12:23:23,created attachment error in website antcomp...,NaN
3,519449.0,PDE,UI,VERIFIED,FIXED,Problem with KEY_NAME,2017-08-03 03:21:28,in product pluginsection recommendedbundles m...,NaN
4,519450.0,JDT,UI,RESOLVED,FIXED,Problem with KEY_NAME,2017-07-13 09:43:02,in renametyperefactoring another type the tem...,NaN
...,...,...,...,...,...,...,...,...,...
46259,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,it should be possible to update a text file us...,NaN
46260,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,in the case of switchlabeledrule compiler a...,NaN
46261,565646.0,Community,CI-Jenkins,CLOSED,DUPLICATE,help needed in ensuring that https://ci.eclips...,2020-08-14 09:01:13,hi are there any memory cpu usage reports fo...,565863.0
46262,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,created attachment downloads for atom vscode...,NaN


In [5]:
Eclipse_data =  Eclipse_data.rename(columns={"Bug_ID": "Bug ID"})
Eclipse_data

,Bug ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
0,518088.0,Web Tools,Web Standard Tools,RESOLVED,INVALID,search widget is not working,2017-06-10 14:42:10,search widget is not working,NaN
1,546444.0,z_Archived,PDT,CLOSED,NOT_ECLIPSE,Bug,2020-05-14 11:25:40,created attachment eclipse hello world,NaN
2,533893.0,Platform,Website,RESOLVED,INVALID,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,2018-04-22 12:23:23,created attachment error in website antcomp...,NaN
3,519449.0,PDE,UI,VERIFIED,FIXED,Problem with KEY_NAME,2017-08-03 03:21:28,in product pluginsection recommendedbundles m...,NaN
4,519450.0,JDT,UI,RESOLVED,FIXED,Problem with KEY_NAME,2017-07-13 09:43:02,in renametyperefactoring another type the tem...,NaN
...,...,...,...,...,...,...,...,...,...
46259,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,it should be possible to update a text file us...,NaN
46260,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,in the case of switchlabeledrule compiler a...,NaN
46261,565646.0,Community,CI-Jenkins,CLOSED,DUPLICATE,help needed in ensuring that https://ci.eclips...,2020-08-14 09:01:13,hi are there any memory cpu usage reports fo...,565863.0
46262,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,created attachment downloads for atom vscode...,NaN


In [6]:
#Dropping duplicate entities
Eclipse_data = Eclipse_data.drop_duplicates()
Eclipse_data = Eclipse_data.dropna(axis=0, subset=['Bug ID'])
Eclipse_data

,Bug ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
0,518088.0,Web Tools,Web Standard Tools,RESOLVED,INVALID,search widget is not working,2017-06-10 14:42:10,search widget is not working,NaN
1,546444.0,z_Archived,PDT,CLOSED,NOT_ECLIPSE,Bug,2020-05-14 11:25:40,created attachment eclipse hello world,NaN
2,533893.0,Platform,Website,RESOLVED,INVALID,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,2018-04-22 12:23:23,created attachment error in website antcomp...,NaN
3,519449.0,PDE,UI,VERIFIED,FIXED,Problem with KEY_NAME,2017-08-03 03:21:28,in product pluginsection recommendedbundles m...,NaN
4,519450.0,JDT,UI,RESOLVED,FIXED,Problem with KEY_NAME,2017-07-13 09:43:02,in renametyperefactoring another type the tem...,NaN
...,...,...,...,...,...,...,...,...,...
46259,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,it should be possible to update a text file us...,NaN
46260,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,in the case of switchlabeledrule compiler a...,NaN
46261,565646.0,Community,CI-Jenkins,CLOSED,DUPLICATE,help needed in ensuring that https://ci.eclips...,2020-08-14 09:01:13,hi are there any memory cpu usage reports fo...,565863.0
46262,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,created attachment downloads for atom vscode...,NaN


## On complete Eclipse data

In [11]:
Eclipse_data["Description"]= Eclipse_data["Description"].str.replace("fixed in HEAD", "", case = False)
Eclipse_data["Description"]= Eclipse_data["Description"].str.replace("has been marked as readonly", " ", case = False)

In [7]:
#Text Cleaning round 1 (removing punctutions)

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
#     print(text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\w*\f\w*', '', text)
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\[.*]\)', '', text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [14]:
# Eclipse_data = Eclipse_data.drop(columns=['Unnamed: 0'])
Eclipse_data = Eclipse_data.dropna(axis=0, subset=['Description'])

In [15]:
Eclipse_data['Description'] = Eclipse_data['Description'].apply(clean_text_round1)

In [8]:
# Apply a second round of cleaning (removing punctuations)
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [17]:
Eclipse_data['Description'] = Eclipse_data['Description'].apply(clean_text_round2)

In [9]:
#Lemmatization
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [19]:
Eclipse_data['Description'] = Eclipse_data['Description'].map(preprocess)

In [20]:
Eclipse_data['Duplicate_Bug_Ids'].isnull().sum()

43015

In [21]:
#Saving all the duplicate reports into a csv file as a testing set
duplicate_reports = Eclipse_data.dropna(axis=0, subset=['Duplicate_Bug_Ids'])
duplicate_reports.reset_index(drop=True)
duplicate_reports.to_csv('Eclipse_duplicate_reports.csv')

In [22]:
#Seperating all the master reports into a dataframe
master_reports = Eclipse_data[Eclipse_data.isnull().any(axis=1)]
master_reports.reset_index(drop=True)

,Bug ID,Product,Component,Status,Resolution,Summary,Changed,Description,Duplicate_Bug_Ids
0,518088.0,Web Tools,Web Standard Tools,RESOLVED,INVALID,search widget is not working,2017-06-10 14:42:10,"[search, widget, work]",NaN
1,546444.0,z_Archived,PDT,CLOSED,NOT_ECLIPSE,Bug,2020-05-14 11:25:40,"[create, attachment, eclipse]",NaN
2,533893.0,Platform,Website,RESOLVED,INVALID,AntCompareCVSDebugDocIDEIncubatorPMCRelengReso...,2018-04-22 12:23:23,"[create, attachment, website, assistance]",NaN
3,519449.0,PDE,UI,VERIFIED,FIXED,Problem with KEY_NAME,2017-08-03 03:21:28,"[product, pluginsection, message, template, fo...",NaN
4,519450.0,JDT,UI,RESOLVED,FIXED,Problem with KEY_NAME,2017-07-13 09:43:02,"[template, reference, semantics, preserve, pro...",NaN
...,...,...,...,...,...,...,...,...,...
43010,537916.0,Platform,UI,VERIFIED,FIXED,org.eclipse.e4.ui.workbench.lifecycle should b...,2018-09-05 04:29:00,"[output, example, wizard, import, annotations,...",NaN
43011,544461.0,Sirius,Server,CLOSED,WONTFIX,Add support for the update of text files,2021-01-06 05:29:02,"[possible, update, graphql]",NaN
43012,545249.0,JDT,UI,RESOLVED,FIXED,[12] AST View should not show the implicit bre...,2019-05-24 06:06:54,"[compiler, implicit, statement, geneerate, req...",NaN
43013,568788.0,z_Archived,Tycho,RESOLVED,FIXED,Support new format .tgz in tycho-p2-director:a...,2021-04-28 16:52:10,"[create, attachment, download, vscode, eclipse...",NaN


In [23]:
master_reports.Description.isna().sum()

0

In [24]:
#Removing empty master_reports after data cleaning process
mr = pd.DataFrame()
count = 0
for i in range(len(master_reports)):
    if(len(master_reports.Description.iloc[i])>2):
        mr= mr.append(master_reports.iloc[[i]])
        count=+1

In [25]:
master_reports.to_csv('Eclipse_master_reports.csv')

In [7]:
#importing the CSV file of the master reports from google drive into a dataframe
master_reports = pd.read_csv('Eclipse_master_reports.csv')
master_reports = master_reports.drop(columns=['Unnamed: 0'])

In [10]:
master_reports['Description'] = master_reports['Description'].map(preprocess)

In [13]:
#Creating a dictionary using gensim library
dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [29]:
#Print top 20 words from the dictionary
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

0 search
1 widget
2 attachment
3 create
4 eclipse
5 assistance
6 website
7 contain
8 follow
9 message
10 product
11 recommend
12 template
13 preserve
14 proceed
15 reference
16 semantics
17 similar
18 templates
19 internal
20 properties


In [14]:
#Creating BoW using the the dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]

In [31]:
#Printing the BoW for single document
bow_doc_8 = bow_corpus[8]
for i in range(len(bow_doc_8)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_8[i][0], 
                                               dictionary[bow_doc_8[i][0]], 
bow_doc_8[i][1]))

Word 4 ("eclipse") appears 1 time.
Word 25 ("string") appears 1 time.
Word 27 ("belong") appears 1 time.
Word 28 ("filter") appears 1 time.
Word 29 ("gitlab") appears 1 time.
Word 30 ("ignore") appears 1 time.
Word 31 ("locale") appears 1 time.
Word 32 ("nightly") appears 1 time.
Word 33 ("request") appears 1 time.
Word 34 ("technology") appears 1 time.
Word 35 ("translate") appears 1 time.
Word 36 ("translations") appears 1 time.


In [15]:
# open a file, where you stored the pickled data
f= open('Eclipse_bow_corpus.pickle', 'wb')

# dump information to that file
pickle.dump(bow_corpus, f)

In [16]:
#open a file, where you stored the pickled data
file = open('Eclipse_dictionary.pickle', 'wb')

# dump information to that file
pickle.dump(dictionary, file)

In [34]:
#Preparing the parameters for LDA model
corpus = bow_corpus
no_of_topics = 10
dictionary = dictionary
p = 20
k = 2
epochs = 100

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=no_of_topics, id2word=dictionary, passes=p, workers=k, iterations=epochs)

In [ ]:
# save model to disk (no need to use pickle module)
lda_model.save('Eclipse_lda_model.model')

In [36]:
# Printing the topics and the propability distributions of words in those topics
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.043*"method" + 0.037*"remove" + 0.035*"string" + 0.034*"public" + 0.032*"return" + 0.019*"condition" + 0.018*"static" + 0.017*"object" + 0.015*"boolean" + 0.015*"primitive"
Topic: 1 
Words: 0.035*"server" + 0.022*"commit" + 0.020*"branch" + 0.019*"jenkins" + 0.018*"gerrit" + 0.017*"access" + 0.017*"change" + 0.016*"request" + 0.015*"repository" + 0.015*"eclipse"
Topic: 2 
Words: 0.135*"project" + 0.041*"eclipse" + 0.017*"create" + 0.016*"release" + 0.016*"license" + 0.015*"account" + 0.014*"process" + 0.014*"github" + 0.013*"instance" + 0.011*"archive"
Topic: 3 
Words: 0.239*"eclipse" + 0.100*"widgets" + 0.043*"display" + 0.032*"sendevent" + 0.032*"widget" + 0.028*"runtime" + 0.024*"papyrus" + 0.022*"command" + 0.015*"execute" + 0.012*"equinox"
Topic: 4 
Words: 0.301*"eclipse" + 0.060*"internal" + 0.023*"thread" + 0.020*"product" + 0.019*"message" + 0.017*"viewers" + 0.017*"resources" + 0.016*"resource" + 0.013*"report" + 0.013*"exception"
Topic: 5 
Words: 0.042*"cre

In [37]:
#Let's evaluate the model using Perplexity and Coherence Bag of words- Title

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=Eclipse_data['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.68935491140499

Coherence Score:  0.6149586302227567


In [40]:
# for visualization of LDA
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
vis

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/users/grad/umukherjee/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.312968 -0.019421       1        1  19.470807
5     -0.225669 -0.107294       2        1  17.020853
7      0.047062 -0.110922       3        1  10.363683
4      0.222769 -0.049164       4        1   9.589214
3      0.279886 -0.111217       5        1   9.099646
1     -0.150224 -0.045368       6        1   8.010928
8     -0.215944 -0.013517       7        1   7.884258
0     -0.196276  0.111964       8        1   6.912172
2     -0.156329 -0.055467       9        1   6.493328
6      0.081757  0.400407      10        1   5.155111, topic_info=           Term           Freq          Total Category  logprob  loglift
4       eclipse  161246.000000  161246.000000  Default  30.0000  30.0000
19     internal   58770.000000   58770.000000  Default  29.0000  29.0000
24      project   14471.000000   14471.000000  Default  28.0000  28.0000
1246    widgets   12930.000000   12930.000000  Default  27.0000  27.0000
1229  workbench   11536.000000   11536.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
715      module     729.987262    2455.858265  Topic10  -4.5077   1.7520
19     internal    1984.188405   58770.229773  Topic10  -3.5078  -0.4232
1646   launcher     797.648740    5853.351334  Topic10  -4.4191   0.9721
129      review     515.012597    1212.263459  Topic10  -4.8566   2.1091
4       eclipse     615.842068  161246.268229  Topic10  -4.6778  -2.6025

[657 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
67        1  0.326659     access
67        2  0.049636     access
67        4  0.067307     access
67        5  0.053274     access
67        6  0.461273     access
...     ...       ...        ...
66        7  0.071184  workspace
66       10  0.008021  workspace
2646      8  0.999543    wrapper
1468      1  0.996857     xerces
3175      4  0.992602     xxxxxx

[1317 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 8, 5, 4, 2, 9, 1, 3, 7])

In [41]:
#Creating 10 empty clusters and pushing master reports in each of them based on topic modeling and saving them in individual csv file
for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
# open a file, where you stored the pickled data
f = open('Eclipse_bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('Eclipse_dictionary.pickle', 'rb')
dictionary=pickle.load(file)

# later on, load trained model from file
lda_model =  models.LdaModel.load('Eclipse_lda_model.model')

In [44]:
#Import all the clusters from the drive 
for c in range(10):
    exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
    exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
    exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
#creating a corpus for Word2Vec and FastText models
for i in range(10):
    exec('sent_{} = []'.format(i))
    exec('x= topic_{}'.format(i))
    for j in range(len(x)):
        exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
for sent in range(10):
    exec('print(len(sent_{}))'.format(sent))

3632
4803
4454
934
1726
14878
1272
4920
4693
1685


/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
#Training GloVe model for each cluster
for cluster in range(10):
    vector_size = 100
    exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
    exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
    exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
    exec('glove{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
    exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
    exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

    #Save the all the models in individual file
    exec('path = get_tmpfile("glove{}.model")'.format(cluster))
    exec('glove{}.save("glove{}.model")'.format(cluster, cluster))

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Performing 1000 training epochs with 3 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epo

## Classification

In [20]:
# importing all the clusters created using LDA based topic modeling
for c in range(10):
    exec('topic_{} = pd.read_csv("topic_{}.csv")'.format(c,c))
    exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
    exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [21]:
#import the duplicate reports for testing purpose
test = pd.read_csv('Eclipse_duplicate_reports.csv')
test = test.drop(columns=['Unnamed: 0'])

In [22]:
# preprocessing textually similar and dissimilar duplicates to evaluate
def lemmatize2(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess2(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize2(token))
    return result

#import the duplicate reports for testing purpose
test_sim = pd.read_csv('eclipse_textually_similar_duplicates.csv')
# test_sim = test_sim.drop(columns=['Unnamed: 0'])

#import the duplicate reports for testing purpose
test_dissim = pd.read_csv('eclipse_textually_dissimilar_duplicates.csv')
# test_dissim = test_dissim.drop(columns=['Unnamed: 0']/)

test_sim['Description']= test_sim['Description'].fillna('').astype(str).map(preprocess2)
test_dissim['Description']= test_dissim['Description'].fillna('').astype(str).map(preprocess2)

In [27]:
def lemmatize2(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess2(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [28]:
test['Description']= test['Description'].fillna('').astype(str).map(preprocess2)

In [47]:
for mod in range(10):
    #import all the trained GloVe models
    exec('glove{} = Glove.load("glove{}.model")'.format(mod, mod))

In [29]:
#This will return the index of cluster in which the master report of duplicate report may reside
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= lda_model[vec_bow]
    topic = np.asarray(x)
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    return sim

In [30]:
# Convert multiple word embeddings into a single document vector by averaging the word embeddings by GloVe model

def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[model.wv.key_to_index[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    

def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.wv.key_to_index)
    if(any(isinstance(i, list) for i in corpus)):
        features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
        return np.array(features)
    else:
          features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
    return np.array(features)

In [32]:
# Similarity between two feature vectors using the average of cosine similarity and euclidean similarity
def sim(vec1, vec2): 
    sim1 = 1/(1+distance.euclidean(vec1, vec2))
    sim2 = cosine_similarity(vec1, vec2)
    sim=(sim1+sim2)/2 
    return sim

In [33]:
# Different kinds of fusion of two master report feature vectors and two duplicate report feature vectors
def fusion(vec1, vec2, vec3, vec4, fusion_no):

    # fusion_no = 1 : concatenation of the vectors
    if (fusion_no == '1'):
        master = np.concatenate((vec1, vec2), axis=1)
        vec_duplicate = np.concatenate((vec3, vec4), axis=0)
        vec_duplicate=[vec_duplicate]
        return vec_duplicate, master

    #fusion_no = 2 : average of the vectors
    elif (fusion_no == '2'):
        vec3 = [vec3]
        vec4 = [vec4]
        avg1 = (np.add(vec1, vec2))/2
        avg2 = (np.add(vec3, vec4))/2
        return avg2, avg1

    #fusion_no = 3 : Dimensionality reduction using PCA on concatenation of the vectors
    elif (fusion_no == '3'):
        master = np.concatenate((vec1, vec2), axis=1)
        pca = PCA(n_components=100)
        avg_fit = pca.fit(master)
        master = pca.transform(master)
        vec_duplicate = np.concatenate((vec3, vec4), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master

    #fusion_no = 3 : Dimensionality reduction using PCA on average of the vectors
    elif (fusion_no == '4'):
        vec3 = [vec3]
        vec4 = [vec4]
        avg1 = (np.add(vec1, vec2))/2
        pca = PCA(n_components=100)
        avg_fit = pca.fit(avg1)
        master = pca.transform(avg1)
        avg2 = (np.add(vec3, vec4))/2
        vec_duplicate = pca.transform(avg2)
        return vec_duplicate, master

    else:
        raise ValueError('Invalid value for fusion')

In [34]:
# creation of feature vectors by multimodality feature extraction
def feature_vectors_multi_modality(DR, corpus, model1, model2, fusion_no):
    master_glove2 = averaged_word_vectorizer_glove(corpus=sent, model=model2, num_features=100)
    vec_duplicate2 = averaged_word_vectorizer_glove(corpus=DR, model=model2, num_features=100)
    vec_duplicate , master= fusion(master_glove2,vec_duplicate2, fusion_no)
    return vec_duplicate,master

In [35]:
# creation of feature vectors by singlemodality feature extraction
def feature_vectors_single_modality(DR, corpus, model1):
    master = averaged_word_vectorizer_glove(corpus=sent, model=model1, num_features=100)

    vec_duplicate = averaged_word_vectorizer_glove(corpus=DR, model=model1, num_features=100)

    vec_duplicate = [vec_duplicate]

    return vec_duplicate, master

In [38]:
test = test.rename(columns={'Bug ID':'Bug_ID'})

In [39]:
def compare_topn(model1, cluster, sent, DR, topn, modal):
    similarity=[]
    if (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality(DR, sent, model1)

    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)

        similarity.append(unified_sim)
    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1][1:]
    return(max_similar_reports)

## LDA and GLOVE

## on testset

In [40]:
#Returns Top-N master reports
dup_id = test.Duplicate_Bug_Ids
def compare_topn2(model1, cluster, sent, DR, topn, modal):
    similarity=[]
    if (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality2(DR, sent, model1)

    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)

        similarity.append(unified_sim)
    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1]

    return(max_similar_reports)

In [41]:
# creation of feature vectors by singlemodality feature extraction
def feature_vectors_single_modality2(DR, corpus, model1):
    master = averaged_word_vectorizer_glove2(corpus=sent, model=model1, num_features=100)

    vec_duplicate = averaged_word_vectorizer_glove2(corpus=DR, model=model1, num_features=100)

    vec_duplicate = [vec_duplicate]

    return vec_duplicate, master

In [42]:
def average_word_vectors_glove2(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
def averaged_word_vectorizer_glove2(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
        features = [average_word_vectors_glove2(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
        return np.array(features)
    else:
          features = average_word_vectors_glove2(corpus, model, vocabulary, num_features)
    return np.array(features)

Determining top-n values for finding recall rate @ k

In [43]:
import math

print(math.floor(1/3))
print(math.floor(5/3))
print(math.floor(10/3))
print(math.floor(20/3))
print(math.floor(25/3))
print(math.floor(30/3))
print(math.floor(50/3))
print(math.floor(75/3))
print(math.floor(100/3))

0
1
3
6
8
10
16
25
33


In [94]:
#top1
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [95]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 0.0 %
Time :  3.5976300398508707 min


In [88]:
#top5
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [89]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 8.5 %
Time :  8.932340415318807 min


In [48]:
#top10
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 3                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 10.5 %
Time :  14.373778569698334 min


In [49]:
#top20
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 6                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 12.5 %
Time :  14.662881906827291 min


In [90]:
#top25
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 8                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [91]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 13.0 %
Time :  8.970532806714376 min


In [50]:
#top30
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 10                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 13.5 %
Time :  14.65589241584142 min


In [92]:
#top50
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 16                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [93]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 14.000000000000002 %
Time :  8.748626677195231 min


In [51]:
#top75
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 25                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 15.5 %
Time :  14.803670871257783 min


In [94]:
#top100
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})
        topn = 33                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [95]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 16.5 %
Time :  8.683452804883322 min


# On Textually Similar and Dissimilar datasets

In [52]:
nan_value = float("NaN")
test_sim.replace("", nan_value, inplace=True)
test_sim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)

In [53]:
nan_value = float("NaN")
test_dissim.replace("", nan_value, inplace=True)
test_dissim.dropna(subset = ["Duplicate_Bug_Ids"], inplace=True)

In [54]:
cond = (test_sim['Duplicate_Bug_Ids'] == 'NaN')
cond.unique()

array([False])

In [55]:
cond = (test_dissim['Duplicate_Bug_Ids'] == 'NaN')
cond.unique()

array([False])

In [56]:
test_sim = test_sim.reset_index(drop=True)

In [57]:
test_dissim = test_dissim.reset_index(drop=True)

In [104]:
len(test_sim.Duplicate_Bug_Ids)

342

# Textually Similar

In [96]:
#top 1
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [97]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 0.0 %
Time :  4.175903781255086 min


In [78]:
#top 5
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [79]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 10.0 %
Time :  9.098893300692241 min


In [60]:
import time

In [61]:
#top 10
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 3                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
        
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 13.5 %
Time :  15.691339270273845 min


In [62]:
#top 20
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 6                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
        
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 15.0 %
Time :  15.695339846611024 min


In [80]:
#top 25
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 8                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [81]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 16.0 %
Time :  9.135705828666687 min


In [63]:
#top 30
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 10                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
        
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 16.5 %
Time :  15.739697070916494 min


In [82]:
#top 50
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 16                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [83]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 17.0 %
Time :  9.218970572948455 min


In [64]:
#top 75
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 25                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
        
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 19.0 %
Time :  15.667101391156514 min


In [84]:
#top 100
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_sim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 33                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_sim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [85]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 20.5 %
Time :  9.135262914498647 min


# Textually dissimilar

In [98]:
#top 1
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 1
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [99]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 0.0 %
Time :  6.128852574030558 min


In [86]:
#top 5
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 1                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [87]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 6.5 %
Time :  8.68234672943751 min


In [65]:
#top 10
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 3                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 11.5 %
Time :  15.450162827968597 min


In [66]:
#top 20
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 6                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 13.0 %
Time :  15.409350899855296 min


In [88]:
#top 25
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 8                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [89]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 14.000000000000002 %
Time :  8.692662858963013 min


In [67]:
#top 30
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 10                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 15.0 %
Time :  15.521968499819438 min


In [90]:
#top 50
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 16                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [91]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 15.5 %
Time :  8.654359443982441 min


In [68]:
#top 75
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 25                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Running sample 199Recall Rate : 17.5 %
Time :  15.377263963222504 min


In [92]:
#top 100
vec_acc=[]
t1 = time.time()
no_of_test_samples= int(200)
for i in range(no_of_test_samples):
    print(f'\rRunning sample {i}', end='')
    sample = test_dissim.Description[i] #The test sample (duplicate report)
    n = 3
    max_cluster =sim_with_clusters_lda_topn(sample, n)
    v=[]
#     print(i)
    for max in max_cluster:
        exec('cluster = topic_{}'.format(max))              #The predicted cluster
#         exec('model1 = ftmodel{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
        exec('model1 = glove{}'.format(max))                #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
        exec('sent = topic_{}.Description'.format(max))     #The vocabulary for the predicted cluster
        cluster = cluster.rename(columns={'Bug ID':'Bug_ID'})

        topn = 33                                          #The number of predicted master report for single predicted cluster
        fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
        modal = 'single'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
         #This will return the Top-N predicted master reports
        max_sim = compare_topn2(model1,cluster, sent, sample, topn, modal)
        t2 = time.time()

        #Comparing the predicted value to the ground truth
        for num in max_sim:
            if (cluster.Bug_ID[num] == test_dissim.Duplicate_Bug_Ids[i]):
                v.append("1")
            else:
                v.append("0")

    if(all(x==v[0] for x in v)):
        vec_acc.append("0")
    else:
        vec_acc.append("1")

Running sample 199

In [93]:
#Evaluating the performance by Recall Rate
sum = 0
for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

Recall Rate : 18.5 %
Time :  8.76777701775233 min
